In [1]:
import torch
from torch import nn
from model import TextRNN
from cnews_loader import read_vocab, read_category, process_file
import numpy as np

['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
x_train= [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]


In [9]:
vocab_file = 'cnews.vocab.txt'
test_file = 'cnews.test.txt'
val_file = 'cnews.val.txt'
train_file = 'cnews.train.txt'
costs = []
min_loss = float('inf')
min_loss = 0.
# 定义模型以及训练的反向传播
def train():
    model = TextRNN().cuda()
    Loss = nn.MultiLabelSoftMarginLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    best_val_acc = 0
    min_loss = 0.
    early_stop = 0
    for epoch in range(4):
        losses = []
        train_correct, train_total = 0., 0.
        print('epoch=', epoch)
        for step, (x_batch, y_batch) in enumerate(train_loader):
            x = x_batch.cuda()
            y = y_batch.cuda()
            out = model(x)
            loss = Loss(out, y)
            print('loss=', loss)
            losses.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_pred = torch.argmax(out, 1).data.cpu().numpy()
            train_label = torch.argmax(y, 1).data.cpu().numpy()
            train_correct += (train_pred == train_label).sum()
            train_total += len(train_label)
            accuracy = np.mean((torch.argmax(out,1) == torch.argmax(y,1)).cpu().numpy())
            print('accuracy:', accuracy)
        meanloss = np.mean(losses)
        costs.append(meanloss)    
        if (epoch + 1) % 5 == 0:
            train_acc = train_correct / train_total
            #textrnn.train(False)
            # 验证集预测
            val_correct, val_total = 0., 0.
            for step, (x_batch, y_batch) in enumerate(val_loader):
                x = x_batch.cuda()
                y = y_batch.cuda()
                out = model(x)
                val_pred = torch.argmax(val_output, 1)
                val_pred_arr = val_pred.data.cpu().numpy()
                y_val_arr = torch.argmax(y, 1).data.cpu().numpy()
                # 准确率
                val_correct += (val_pred_arr == y_val_arr).sum()
                val_total += len(y_val_arr)
                
                accuracy = np.mean((torch.argmax(out,1) == torch.argmax(y,1)).cpu().numpy())
            val_acc = val_correct / val_total
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(textrnn, 'textRNN1.pt')
        if meanloss < min_loss:
            min_loss = meanloss
            early_stop = 0
        else:
            early_stop += 1
        if early_stop > 5:
            print(f"loss连续{epoch}个epoch未降低, 停止循环")
            break

In [10]:
from torch import optim
# 获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()
print(categories)
# 获取训练文本中所有出现过的字及其所对应的id
words, word_to_id = read_vocab('cnews.vocab.txt')
#print(words)
#print(word_to_id)
#print(word_to_id)
#获取字数
vocab_size = len(words)
import torch.utils.data as Data
# 数据加载及分批
# 获取训练数据每个字的id和对应标签的one-hot形式
x_train, y_train = process_file('cnews.train.txt', word_to_id, cat_to_id, 600)
print('x_train=', x_train)
x_val, y_val = process_file('cnews.val.txt', word_to_id, cat_to_id, 600)

cuda = torch.device('cuda')
x_train, y_train = torch.LongTensor(x_train), torch.Tensor(y_train)
x_val, y_val = torch.LongTensor(x_val), torch.Tensor(y_val)
train_dataset = Data.TensorDataset(x_train,y_train)
train_loader = Data.DataLoader(dataset=train_dataset, batch_size = 128, shuffle =True)
val_dataset = Data.TensorDataset(x_val,y_val)
val_loader = Data.DataLoader(dataset=val_dataset, batch_size = 1000)
train()

['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
x_train= [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]
epoch= 0
loss= tensor(0.7347, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.0703125
loss= tensor(0.7342, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.0859375
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.15625
loss= tensor(0.7340, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.1328125
loss= tensor(0.7342, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.140625
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.1171875
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.09375
loss= tensor(0.7348, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.0703125
loss= tensor(0.7338, device='cuda:

accuracy: 0.34375
loss= tensor(0.7221, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.2578125
loss= tensor(0.7185, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.328125
loss= tensor(0.7189, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3203125
loss= tensor(0.7199, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.328125
loss= tensor(0.7196, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3359375
loss= tensor(0.7215, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.2734375
loss= tensor(0.7199, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.296875
loss= tensor(0.7211, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.296875
loss= tensor(0.7230, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.296875
loss= tensor(0.7227, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.296875
loss= tensor(0.7173, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3046875
loss= tensor(0.7189, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.

accuracy: 0.3203125
loss= tensor(0.7128, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.375
loss= tensor(0.7223, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.2421875
loss= tensor(0.7182, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.359375
loss= tensor(0.7124, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.390625
loss= tensor(0.7155, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3671875
loss= tensor(0.7167, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3359375
loss= tensor(0.7184, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3359375
loss= tensor(0.7155, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3671875
loss= tensor(0.7124, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.40625
loss= tensor(0.7143, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3984375
loss= tensor(0.7191, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.328125
loss= tensor(0.7221, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.2

accuracy: 0.3359375
loss= tensor(0.7158, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3203125
loss= tensor(0.7186, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3203125
loss= tensor(0.7178, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.359375
loss= tensor(0.7164, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.328125
loss= tensor(0.7213, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3203125
loss= tensor(0.7148, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.359375
loss= tensor(0.7152, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.359375
loss= tensor(0.7177, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3125
loss= tensor(0.7151, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.390625
loss= tensor(0.7138, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.34375
loss= tensor(0.7189, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.328125
loss= tensor(0.7175, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.328

accuracy: 0.265625
loss= tensor(0.7155, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3671875
loss= tensor(0.7154, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.375
loss= tensor(0.7183, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3125
loss= tensor(0.7151, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.375
loss= tensor(0.7158, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3515625
loss= tensor(0.7175, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3203125
loss= tensor(0.7189, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3203125
loss= tensor(0.7096, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.4296875
loss= tensor(0.7194, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.296875
loss= tensor(0.7174, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3515625
loss= tensor(0.7234, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.234375
loss= tensor(0.7156, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.335937

...

accuracy: 0.4140625
loss= tensor(0.7149, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3359375
loss= tensor(0.7247, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.2421875
loss= tensor(0.7163, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.375
loss= tensor(0.7185, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3203125
loss= tensor(0.7113, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.421875
loss= tensor(0.7133, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.34375
loss= tensor(0.7140, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3671875
loss= tensor(0.7190, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3203125
loss= tensor(0.7171, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3359375
loss= tensor(0.7107, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.40625
loss= tensor(0.7194, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.296875
loss= tensor(0.7179, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.35

accuracy: 0.359375
loss= tensor(0.7161, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.34375
loss= tensor(0.7111, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.421875
loss= tensor(0.7121, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.40625
loss= tensor(0.7117, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.421875
loss= tensor(0.7154, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3671875
loss= tensor(0.7163, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.359375
loss= tensor(0.7127, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.40625
loss= tensor(0.7153, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.375
loss= tensor(0.7147, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.390625
loss= tensor(0.7151, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.359375
loss= tensor(0.7134, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.390625
loss= tensor(0.7143, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3671875
l

accuracy: 0.3359375
loss= tensor(0.7157, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.34375
loss= tensor(0.7120, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3984375
loss= tensor(0.7150, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3515625
loss= tensor(0.7099, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.4296875
loss= tensor(0.7126, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.375
loss= tensor(0.7062, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.4609375
loss= tensor(0.7141, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.390625
loss= tensor(0.7073, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.453125
loss= tensor(0.7084, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.421875
loss= tensor(0.7094, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.4375
loss= tensor(0.7114, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.390625
loss= tensor(0.7115, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.4375


loss= tensor(0.7131, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3828125
loss= tensor(0.7135, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3671875
loss= tensor(0.7141, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.359375
loss= tensor(0.7106, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.4140625
loss= tensor(0.7164, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.390625
loss= tensor(0.7135, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3828125
loss= tensor(0.7226, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.2890625
loss= tensor(0.7158, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.328125
loss= tensor(0.7080, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.453125
loss= tensor(0.7109, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.375
loss= tensor(0.7207, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3125
loss= tensor(0.7161, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3828125
loss= tensor(0.

accuracy: 0.3515625
loss= tensor(0.7096, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.4375
loss= tensor(0.7185, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3046875
loss= tensor(0.7167, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3515625
loss= tensor(0.7131, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.359375
loss= tensor(0.7107, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3828125
loss= tensor(0.7137, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.359375
loss= tensor(0.7124, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3984375
loss= tensor(0.7140, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.40625
loss= tensor(0.7161, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.3359375
loss= tensor(0.7110, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.4296875
loss= tensor(0.7132, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.390625
loss= tensor(0.7143, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy: 0.